In [1]:
# Enable module reloading
%load_ext autoreload
%autoreload 2

%load_ext tensorboard

import datetime
import os
import sys
sys.path.append('..')

import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import time;

from tensorflow.keras import initializers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Input, Dense, Flatten, concatenate
from tensorflow.keras.utils import to_categorical
from src.data.load_data import *

In [2]:
df = load_dataset(filename='../data/Jester-Dataset-ratings.csv')
user_ids, joke_ids, ratings = get_data(df, batch_size=20000)
# batch_size = 20000
        
# matrix = df[0:batch_size].pivot(index='USER_ID', columns='JOKE_ID', values='Rating').fillna(0)
# user_ids = matrix.index.tolist()
# joke_ids = matrix.columns.tolist()
# ratings = matrix.values.flatten().tolist()
# ratings = np.array(cur_df.pivot(index='USER_ID', columns='JOKE_ID', values='Rating').fillna(0)).tolist()

In [3]:
user_onehot = to_categorical(user_ids)
joke_onehot = to_categorical(joke_ids)

In [4]:
n_users = len(user_onehot[0])
n_jokes = len(joke_onehot[0])
# n_users = len(np.unique(user_ids))
# n_jokes = len(np.unique(joke_ids))

In [5]:
num_classes = n_users if n_users > n_jokes else n_jokes
user_onehot = to_categorical(user_ids, num_classes=num_classes)
joke_onehot = to_categorical(joke_ids, num_classes=num_classes)

In [6]:
num_classes

511

In [7]:
vector_concat = np.concatenate([user_onehot, joke_onehot], axis=1)

In [8]:
np.array(vector_concat)

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [9]:
np.array(list(zip(user_ids, joke_ids)))

array([[  1,   5],
       [  1,   7],
       [  1,   8],
       ...,
       [510,  68],
       [510,  24],
       [510,  25]])

In [19]:
from src.models.model_GMF import JokeRecommender

model = JokeRecommender(n_users, n_jokes)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.01), 
    loss='mean_squared_error', 
#     learning_rate=0.01,
    metrics=['accuracy']
)

tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir=os.path.join("../logs", str(time.time())),
    histogram_freq=1)

model.fit(
    x=np.array(vector_concat),
    y=np.array(ratings).astype('float32'),
    batch_size=100,
    epochs=20,
    callbacks=[tensorboard_callback],
    validation_split=0.2
)

Train on 16000 samples, validate on 4000 samples
Epoch 1/20
16000/16000 [==============================] - 5s 291us/sample - loss: 33.0535 - accuracy: 0.0018 - val_loss: 33.1055 - val_accuracy: 7.5000e-04
Epoch 2/20
16000/16000 [==============================] - 3s 203us/sample - loss: 32.9582 - accuracy: 0.0018 - val_loss: 33.0790 - val_accuracy: 7.5000e-04
Epoch 3/20
 7700/16000 [=============>................] - ETA: 1s - loss: 33.1658 - accuracy: 0.0018

KeyboardInterrupt: 

In [ ]:
result = model.predict(np.array(vector_concat))

In [ ]:
pd.DataFrame(result).describe()

In [11]:
np.column_stack((result, ratings))[:200]

NameError: name 'result' is not defined

In [16]:
np.array(ratings).astype('float32')

array([ 0.219, -9.281, -9.281, ...,  3.844,  7.656,  5.469], dtype=float32)